In [9]:
import tensorflow as tf 
from tensorflow import keras
from glob import glob
import numpy as np

import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

from model import RetinaNet,get_backbone
from loss import RetinaNetLoss 
from dataloader import FaceMask, preprocess_data
from box import LabelEncoder

In [18]:
num_classes = 2
classes_name = ['face','mask']
batch_size = 2
model_dir = "retinanet/"
label_encoder = LabelEncoder()

learning_rates = [2.5e-06, 0.000625, 0.00125, 0.0025, 0.00025, 2.5e-05]
learning_rate_boundaries = [125, 250, 500, 240000, 360000]
learning_rate_fn = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=learning_rate_boundaries, values=learning_rates
)


In [11]:
resnet50_backbone = get_backbone()
loss_fn = RetinaNetLoss(num_classes)
model = RetinaNet(num_classes, resnet50_backbone)

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, "weights" + "_epoch_{epoch}"),
        monitor="loss",
        save_best_only=False,
        save_weights_only=True,
        verbose=1,
    )
]

In [12]:

optimizer = tf.optimizers.SGD(learning_rate=learning_rate_fn, momentum=0.9)
model.compile(loss=loss_fn, optimizer=optimizer)

In [13]:
train_img_paths = sorted(glob('./facedataset/images/train/*.jpg'))
train_labels_paths = sorted(glob('./facedataset/labels/train/*.txt'))
val_img_paths = sorted(glob('./facedataset/images/val/*.jpg'))
val_labels_paths = sorted(glob('./facedataset/labels/val/*.txt'))


train_gen =FaceMask(train_img_paths, train_labels_paths)
train_dataset = tf.data.Dataset.from_generator(
     lambda: train_gen,
     output_types=(tf.uint8,tf.float32,tf.int32),
     output_shapes=((None, None, 3),(None, 4),(None,)))

In [14]:
train_img_paths = sorted(glob('./facedataset/images/train/*.jpg'))
train_labels_paths = sorted(glob('./facedataset/labels/train/*.txt'))
val_img_paths = sorted(glob('./facedataset/images/val/*.jpg'))
val_labels_paths = sorted(glob('./facedataset/labels/val/*.txt'))


train_gen =FaceMask(train_img_paths, train_labels_paths)
train_dataset = tf.data.Dataset.from_generator(
     lambda: train_gen,
     output_types=(tf.uint8,tf.float32,tf.int32),
     output_shapes=((None, None, 3),(None, 4),(None,)))

autotune = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=autotune)
train_dataset = train_dataset.shuffle(8 * batch_size)
train_dataset = train_dataset.padded_batch(batch_size=batch_size, padding_values=(0.0, 1e-8, -1), drop_remainder=True)

train_dataset = train_dataset.map(label_encoder.encode_batch, num_parallel_calls=autotune)
train_dataset = train_dataset.apply(tf.data.experimental.ignore_errors())
train_dataset = train_dataset.prefetch(autotune)


val_gen =FaceMask(val_img_paths, val_labels_paths)
val_dataset = tf.data.Dataset.from_generator(
     lambda: val_gen,
     output_types=(tf.uint8,tf.float32,tf.int32),
     output_shapes=((None, None, 3),(None, 4),(None,)))

val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=autotune)
val_dataset = val_dataset.padded_batch(batch_size=batch_size, padding_values=(0.0, 1e-8, -1), drop_remainder=True)

val_dataset = val_dataset.map(label_encoder.encode_batch, num_parallel_calls=autotune)
val_dataset = val_dataset.apply(tf.data.experimental.ignore_errors())
val_dataset = val_dataset.prefetch(autotune)

In [19]:
epochs = 2




model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=callbacks_list,
    verbose=1,
)

Epoch 1/2


ResourceExhaustedError:  OOM when allocating tensor with shape[4,256,288,352] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node RetinaNet/FeaturePyramid/functional_1/conv2_block1_0_conv/Conv2D (defined at d:\fu\retina\model.py:42) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_23005]

Function call stack:
train_function
